# Description

Reads in a domain and problem file, and generates a networkx graph from the entire search space by performing a breadth first search until a solution is found.

## Loading the Problem

In [21]:
from pyperplan import _parse, _ground
import numpy as np
import networkx as nx
import os

Prime number generator later used by hashing function to uniquely represent states

In [22]:
def gen_primes():
    """ Generate an infinite sequence of prime numbers.
    """
    # Maps composites to primes witnessing their compositeness.
    # This is memory efficient, as the sieve is not "run forward"
    # indefinitely, but only as long as required by the current
    # number being tested.
    #
    D = {}
    
    # The running integer that's checked for primeness
    q = 2
    
    while True:
        if q not in D:
            # q is a new prime.
            # Yield it and mark its first multiple that isn't
            # already marked in previous iterations
            # 
            yield q
            D[q * q] = [q]
        else:
            # q is composite. D[q] is the list of primes that
            # divide it. Since we've reached q, we no longer
            # need it in the map, but we'll mark the next 
            # multiples of its witnesses to prepare for larger
            # numbers
            # 
            for p in D[q]:
                D.setdefault(p + q, []).append(p)
            del D[q]
        
        q += 1

Parse the PDDL problem

In [23]:
pddl_dir = "pddl_files"
domain_path = "logistics/43/domain.pddl"
problem_path = "logistics/43/problogistics-6-1.pddl"
problem = _parse(domain_file=os.path.join(pddl_dir, domain_path),problem_file=os.path.join(pddl_dir, problem_path))
task = _ground(problem)

In [24]:
task.facts

{'(at apn1 apt1)',
 '(at apn1 apt2)',
 '(at apn1 pos1)',
 '(at apn1 pos2)',
 '(at obj11 apt1)',
 '(at obj11 apt2)',
 '(at obj11 pos1)',
 '(at obj11 pos2)',
 '(at obj12 apt1)',
 '(at obj12 apt2)',
 '(at obj12 pos1)',
 '(at obj12 pos2)',
 '(at obj13 apt1)',
 '(at obj13 apt2)',
 '(at obj13 pos1)',
 '(at obj13 pos2)',
 '(at obj21 apt1)',
 '(at obj21 apt2)',
 '(at obj21 pos1)',
 '(at obj21 pos2)',
 '(at obj22 apt1)',
 '(at obj22 apt2)',
 '(at obj22 pos1)',
 '(at obj22 pos2)',
 '(at obj23 apt1)',
 '(at obj23 apt2)',
 '(at obj23 pos1)',
 '(at obj23 pos2)',
 '(at tru1 apt1)',
 '(at tru1 apt2)',
 '(at tru1 pos1)',
 '(at tru1 pos2)',
 '(at tru2 apt1)',
 '(at tru2 apt2)',
 '(at tru2 pos1)',
 '(at tru2 pos2)',
 '(in obj11 apn1)',
 '(in obj11 tru1)',
 '(in obj11 tru2)',
 '(in obj12 apn1)',
 '(in obj12 tru1)',
 '(in obj12 tru2)',
 '(in obj13 apn1)',
 '(in obj13 tru1)',
 '(in obj13 tru2)',
 '(in obj21 apn1)',
 '(in obj21 tru1)',
 '(in obj21 tru2)',
 '(in obj22 apn1)',
 '(in obj22 tru1)',
 '(in obj22 

In [25]:
objects = list(problem.objects.keys())
objects

['apn1',
 'apt2',
 'pos2',
 'apt1',
 'pos1',
 'cit2',
 'cit1',
 'tru2',
 'tru1',
 'obj23',
 'obj22',
 'obj21',
 'obj13',
 'obj12',
 'obj11']

In [26]:
predicates = list(problem.domain.predicates.keys())
predicates

['package',
 'truck',
 'airplane',
 'airport',
 'location',
 'in-city',
 'city',
 'at',
 'in']

In [27]:
actions = list(problem.domain.actions.keys())
actions

['load-truck',
 'load-airplane',
 'unload-truck',
 'unload-airplane',
 'drive-truck',
 'fly-airplane']

In [28]:
tokens = actions + objects + predicates
token_mapping = {token: p for token, p in zip(tokens, gen_primes())}
token_mapping

{'load-truck': 2,
 'load-airplane': 3,
 'unload-truck': 5,
 'unload-airplane': 7,
 'drive-truck': 11,
 'fly-airplane': 13,
 'apn1': 17,
 'apt2': 19,
 'pos2': 23,
 'apt1': 29,
 'pos1': 31,
 'cit2': 37,
 'cit1': 41,
 'tru2': 43,
 'tru1': 47,
 'obj23': 53,
 'obj22': 59,
 'obj21': 61,
 'obj13': 67,
 'obj12': 71,
 'obj11': 73,
 'package': 79,
 'truck': 83,
 'airplane': 89,
 'airport': 97,
 'location': 101,
 'in-city': 103,
 'city': 107,
 'at': 109,
 'in': 113}

In [29]:
G = nx.Graph()

In [30]:
def hash_state(state):
    hash = 1
    
    if type(state) == tuple:
        temp = state[0].name[1:-1].split(" ")[0]
        hash *= token_mapping[temp]
        
        for fact in state[1]:
            temp_tokens = fact[1:-1]
            temp_tokens = temp_tokens.split(" ")

            for temp_token in temp_tokens:
                hash *= token_mapping[temp_token]        
        
    else:    
        for fact in state:
            temp_tokens = fact[1:-1]
            temp_tokens = temp_tokens.split(" ")

            for temp_token in temp_tokens:
                hash *= token_mapping[temp_token]
            
    return hash

## Generating Graph

By modifying the BFS function that's already a part of pyperplan, we can just hook in some code that builds up a networkx graph

In [31]:
from collections import deque
import logging

from search import searchspace


def breadth_first_search(planning_task):
    '''
    Searches for a plan on the given task using breadth first search and
    duplicate detection.

    @param planning_task: The planning task to solve.
    @return: The solution as a list of operators or None if the task is
    unsolvable.
    '''
    # counts the number of loops (only for printing)
    G = nx.Graph()
    iteration = 0
    # fifo-queue storing the nodes which are next to explore
    queue = deque()
    queue.append(searchspace.make_root_node(planning_task.initial_state))
    # set storing the explored nodes, used for duplicate detection
    closed = {planning_task.initial_state}
    while queue:
        iteration += 1
        logging.debug("breadth_first_search: Iteration %d, #unexplored=%d"
                      % (iteration, len(queue)))
        # get the next node to explore
        node = queue.popleft()
        # exploring the node or if it is a goal node extracting the plan
        if planning_task.goal_reached(node.state):
            logging.info("Goal reached. Start extraction of solution.")
            logging.info("%d Nodes expanded" % iteration)
            return G
        for operator, successor_state in planning_task.get_successor_states(
                                                                   node.state):
            # duplicate detection
            if successor_state not in closed:
                new_node = searchspace.make_child_node(node, operator,
                                                         successor_state)
                queue.append(new_node)
                G.add_edge(hash_state(node.state), hash_state(new_node.state))
                 # remember the successor state
                # print("node.g: {} | new_node.g: {}".format(node.g, new_node.g))
                closed.add(successor_state)
    logging.info("No operators left. Task unsolvable.")
    logging.info("%d Nodes expanded" % iteration)
    return G

In [32]:
G = breadth_first_search(task)

Since prime number state representation results in ridiculously large numbers, we map these numbers to smaller ones by creating a new graph

In [33]:
new_G = nx.Graph()
node_mapping = {n: i for i, n in enumerate(list(G.nodes))}

for edge in list(G.edges):
    new_G.add_edge(node_mapping[edge[0]], node_mapping[edge[1]])

In [34]:
len(new_G.nodes)

1991

Save output in format required by c++ implementation of node2vec from snap-stanford

In [39]:
graph_dir =  "graphs"
graph_file = os.path.join(graph_dir, os.path.dirname(problem_path))

if not os.path.exists(graph_file):
    os.makedirs(graph_file)
    
graph_file = os.path.join(graph_file, os.path.basename(problem_path).split(".")[0] + ".edgelist")

with open(graph_file, "w") as f:
    for edge in list(new_G.edges):
        f.write("{} {}\n".format(edge[0], edge[1]))